## LLM 모델 활용하여 사용자 리뷰 데이터 긍정부정 구분하는 예시

### *1. 필수 라이브러리 설치 및 로드(+API KEY 설정)*

In [5]:
# !pip install langchain
# !pip install openai
# !pip install langchain_community
# !pip install langchain_core
# !pip install langchain_openai
# !pip install python-dotenv
# !pip install langchain_experimental
# !pip install sentence_transformers
# !pip install faiss-cpu

In [6]:
# import
import pandas as pd
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import CSVLoader, DataFrameLoader
import os

In [7]:
# open API KEY 저장
os.environ['OPEN_API_KEY'] = ''

### *2. 데이터 로드*

In [8]:
df = pd.read_excel(r'3_naver_user_크롤링_user_df_지선.xlsx')
df

,user_name,place_name,place_cate,place_loc,use_method,wait_time,purpose,visit_with,place_review,place_tag,visit_date_list,visit_weekday,visit_count,긍정/부정
0,giblihi,오봉집 석촌,한식,서울특별시 송파구 송파동,포장·배달,바로 입장,일상,연인·배우자,맛있어요 굳굳,'음식이 맛있어요',5월 31일 금요일,금요일,1,긍정
1,giblihi,교차점,돈가스,서울특별시 송파구 석촌동,예약 없이,바로 입장,일상,연인·배우자,개인적으로 송파구에서 제일 맛있는 돈까스 가게예요\n첫 오픈당시부터 맛집포스 풍기더...,"'음식이 맛있어요', '+4\n개의 리뷰 더 있습니다'",5월 22일 수요일,수요일,4,긍정
2,giblihi,현대마트,"슈퍼,마트",서울특별시 송파구 석촌동,정보없음,정보없음,정보없음,정보없음,파인애플 8800원\n오렌지 11000원,정보없음,5월 21일 화요일,화요일,8,중립
3,giblihi,에이라운드헤어,미용실,서울특별시 송파구 석촌동,정보없음,정보없음,정보없음,정보없음,친절해요,"'친절해요', '매장이 청결해요'",5월 19일 일요일,일요일,1,긍정
4,giblihi,써브웨이 석촌역점,샌드위치,서울특별시 송파구 송파동,포장·배달,바로 입장,일상,연인·배우자,맛있어요\n배달료는 사악함,'음식이 맛있어요',5월 19일 일요일,일요일,2,부정
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44977,어른감자,CU 역삼유토빌점,편의점,서울특별시 강남구 역삼동,정보없음,정보없음,정보없음,정보없음,굿,"'친절해요', '상품이 다양해요', '트렌디한 상품이 많아요'",1월 3일 수요일,수요일,4,NaN
44978,어른감자,애슐리퀸즈 뉴코아 평촌점,패밀리레스토랑,경기도 안양시 동안구 호계동,예약 없이,바로 입장,데이트,연인·배우자,고기가 무한리필이라서 양껏 먹고왔습니다\n가격대비 좋아요,"'음식이 맛있어요', '재료가 신선해요', '양이 많아요'",1월 1일 월요일,월요일,1,NaN
44979,어른감자,왕가탕후루 강남역점,"카페,디저트",서울특별시 강남구 역삼동,예약 없이,바로 입장,데이트,연인·배우자,탕후루는 레알 왕가가 진리,'디저트가 맛있어요',12월 31일 일요일,일요일,1,NaN
44980,어른감자,청년떡볶이 경부고속터미널점,떡볶이,서울특별시 서초구 반포동,예약 없이,바로 입장,일상,혼자,터미널 갈때마다 요기 할 겸 어묵 먹는데\n맛있어요 황태국물이 예술,'음식이 맛있어요',12월 31일 일요일,일요일,3,NaN


#### place_cate, place_review, place_tag 전처리

In [9]:
# 제거할 카테고리, 리뷰, 태그
remove_cate = ['동물병원', '아울렛', '종합생활용품', '슈퍼,마트', '사진,스튜디오','미용실','꽃집,꽃배달','패션잡화', '헬스장', '정보없음', '워터파크',
               'GS칼텍스주유소', '편의점', '서점', '안과', '약국', '패션', '캐주얼웨어', '문구,팬시용품', '영화관', '고속도로휴게소', '치과',
               '스포츠용품', '백화점', '푸드트럭', '드럭스토어', '불교용품', '공사,공단', '지하철,전철', '신발',  '인테리어소품', '오락실', '종합패션',
               '헤어악세서리',  '애플리케이션개발', '금융서비스', '기능성화장품',  '은행', '통신', '휴대폰', 'IT서비스', '카셰어링', '인터넷상거래',
               '신용카드', '도시가스', '기업', '도서관', '급식', '대리석제조', '물류센터', '우체국', '출력,복사,제본', '과학기술정보통신부',
               '시민단체', '소프트웨어개발', '지불,결제대행', '내과', '도서,음반,문구', '지방행정기관', '남성정장', '스크린골프장',  '마취통증의학과',
               '한의원', '피부,체형관리', '정신건강의학과', '노래방', '생활협동조합', '민영주차장', '반려동물미용', '타월,수건', 'SK주유소', '병원,의원',
               '왁싱,제모',  'S', 'HD현대오일뱅크주유소', '공영주차장', '장소대여', '면허시험장', '네일아트,네일샵', '메이크업', '캠코더,카메라', '주차장',
               '호텔', '도시,테마공원', '섬', '레저,테마',  '호수,연못,저수지', '산',  '찜질방', '보세의류', '전망대', 'PC방', '하천', '유원지', '방면정보',
               '애견사료', '주유소', '보드카페', '안경원', '화훼재배', '농산물창고', '테마카페', '식물원,수목원', '정형외과']


new_df = df[~df['place_cate'].isin(remove_cate)]  # ~ 연산자는 불리언 시리즈를 반전
new_df = new_df[new_df['place_review']!='정보없음']
new_df = new_df[new_df['place_tag']!='정보없음']
new_df.head(5)

,user_name,place_name,place_cate,place_loc,use_method,wait_time,purpose,visit_with,place_review,place_tag,visit_date_list,visit_weekday,visit_count,긍정/부정
0,giblihi,오봉집 석촌,한식,서울특별시 송파구 송파동,포장·배달,바로 입장,일상,연인·배우자,맛있어요 굳굳,'음식이 맛있어요',5월 31일 금요일,금요일,1,긍정
1,giblihi,교차점,돈가스,서울특별시 송파구 석촌동,예약 없이,바로 입장,일상,연인·배우자,개인적으로 송파구에서 제일 맛있는 돈까스 가게예요\n첫 오픈당시부터 맛집포스 풍기더...,"'음식이 맛있어요', '+4\n개의 리뷰 더 있습니다'",5월 22일 수요일,수요일,4,긍정
4,giblihi,써브웨이 석촌역점,샌드위치,서울특별시 송파구 송파동,포장·배달,바로 입장,일상,연인·배우자,맛있어요\n배달료는 사악함,'음식이 맛있어요',5월 19일 일요일,일요일,2,부정
5,giblihi,올튼,베이커리,서울특별시 송파구 석촌동,포장·배달,바로 입장,일상,정보없음,카페 외부에 붙어있는 크루아상 사진에 홀려 입장했어요\n식사빵 메뉴가 주입니다\n빵...,'친절해요',5월 16일 목요일,목요일,1,부정
7,giblihi,소미연,중식당,서울특별시 송파구 석촌동,예약 없이,바로 입장,일상,연인·배우자,배달로 먹어 본 후 맛있어서 직접 매장에서 먹어보려고 방문했어요\n식사시간 전이라 ...,"'음식이 맛있어요', '매장이 넓어요', '친절해요'",5월 16일 목요일,목요일,2,긍정


In [10]:
# 리뷰의 길이가 짧은 경우 제외시킴
new_df = new_df[new_df['place_review'].apply(lambda x: isinstance(x, str) and len(x.split()) > 1)]
new_df.head(5)

,user_name,place_name,place_cate,place_loc,use_method,wait_time,purpose,visit_with,place_review,place_tag,visit_date_list,visit_weekday,visit_count,긍정/부정
0,giblihi,오봉집 석촌,한식,서울특별시 송파구 송파동,포장·배달,바로 입장,일상,연인·배우자,맛있어요 굳굳,'음식이 맛있어요',5월 31일 금요일,금요일,1,긍정
1,giblihi,교차점,돈가스,서울특별시 송파구 석촌동,예약 없이,바로 입장,일상,연인·배우자,개인적으로 송파구에서 제일 맛있는 돈까스 가게예요\n첫 오픈당시부터 맛집포스 풍기더...,"'음식이 맛있어요', '+4\n개의 리뷰 더 있습니다'",5월 22일 수요일,수요일,4,긍정
4,giblihi,써브웨이 석촌역점,샌드위치,서울특별시 송파구 송파동,포장·배달,바로 입장,일상,연인·배우자,맛있어요\n배달료는 사악함,'음식이 맛있어요',5월 19일 일요일,일요일,2,부정
5,giblihi,올튼,베이커리,서울특별시 송파구 석촌동,포장·배달,바로 입장,일상,정보없음,카페 외부에 붙어있는 크루아상 사진에 홀려 입장했어요\n식사빵 메뉴가 주입니다\n빵...,'친절해요',5월 16일 목요일,목요일,1,부정
7,giblihi,소미연,중식당,서울특별시 송파구 석촌동,예약 없이,바로 입장,일상,연인·배우자,배달로 먹어 본 후 맛있어서 직접 매장에서 먹어보려고 방문했어요\n식사시간 전이라 ...,"'음식이 맛있어요', '매장이 넓어요', '친절해요'",5월 16일 목요일,목요일,2,긍정


In [11]:
len(new_df)

19509

> 1289 rows 에서 655 rows로

In [12]:
# 필요한 칼럼만 들고오기
data = new_df.filter(['user_name','place_review',	'place_tag', 'visit_count'])
data = data.reset_index()
data['index'] = data['index']+1
data

,index,user_name,place_review,place_tag,visit_count
0,1,giblihi,맛있어요 굳굳,'음식이 맛있어요',1
1,2,giblihi,개인적으로 송파구에서 제일 맛있는 돈까스 가게예요\n첫 오픈당시부터 맛집포스 풍기더...,"'음식이 맛있어요', '+4\n개의 리뷰 더 있습니다'",4
2,5,giblihi,맛있어요\n배달료는 사악함,'음식이 맛있어요',2
3,6,giblihi,카페 외부에 붙어있는 크루아상 사진에 홀려 입장했어요\n식사빵 메뉴가 주입니다\n빵...,'친절해요',1
4,8,giblihi,배달로 먹어 본 후 맛있어서 직접 매장에서 먹어보려고 방문했어요\n식사시간 전이라 ...,"'음식이 맛있어요', '매장이 넓어요', '친절해요'",2
...,...,...,...,...,...
19504,44977,어른감자,주 1회 방문중ㅠㅠㅠㅠ맛있어요,"'음식이 맛있어요', '재료가 신선해요', '양이 많아요', '가성비가 좋아요', ...",43
19505,44979,어른감자,고기가 무한리필이라서 양껏 먹고왔습니다\n가격대비 좋아요,"'음식이 맛있어요', '재료가 신선해요', '양이 많아요'",1
19506,44980,어른감자,탕후루는 레알 왕가가 진리,'디저트가 맛있어요',1
19507,44981,어른감자,터미널 갈때마다 요기 할 겸 어묵 먹는데\n맛있어요 황태국물이 예술,'음식이 맛있어요',3


In [13]:
# 할당량 제한이 있으니까 조금만 테스트 (각 유저 별로 1개씩만 뽑기)
def sample(group):
  if len(group) >=2:
    return group.sample(1, replace=False)
  return group

# apply 함수를 사용하여 각 그룹에서 무작위 샘플링
sample_df = data.groupby('user_name').apply(sample).reset_index(drop=True)
sample_df

,index,user_name,place_review,place_tag,visit_count
0,37373,23499,편하게 후다닥 \n유모차 가져가기도 좋구,"'매장이 넓어요', '주차하기 편해요', '아이와 가기 좋아요', '음식이 빨리 나와요'",1
1,27382,4sik9DAD,역시나 시장 \n값좋고 양도 많고 신선하고 좋네요\n마트보다 전통시장입니다\n\n ...,"'신선해요', '양이 많아요', '친절해요', '가성비가 좋아요', '주차하기 편해요'",1
2,34283,Acecream,여름엔 동치미국수도 별미네요 \n비빔국수랑 고기만두까지 함께하니 배도 부르고\n간단...,"'매장이 청결해요', '음식이 맛있어요', '특별한 메뉴가 있어요', '가성비가 좋아요'",8
3,14514,Apricot92,아아 맛도 가격도 굿굿,"'음료가 맛있어요', '특별한 메뉴가 있어요', '친절해요'",4
4,26000,AziA,고기도 구워주구 맛있구 \n아쉽게도 양이 적으니 추가 추가요,'음식이 맛있어요',1
...,...,...,...,...,...
175,20183,헤일리얌,스키야키\n분위기도 좋고 맛도 좋고 다만 가격의 부담이 있네요 예약하고 방문하니 아...,"'메뉴 구성이 알차요', '아늑해요', '대화하기 좋아요'",1
176,29566,홍익2689,음식이 맛있어요,'음식이 맛있어요',1
177,21592,후히후히히,떡볶이를 좀 오래 끓인 담에 먹는게 더 맛있어요,'음식이 맛있어요',3
178,39233,휩쓸어,토핑도 많이 올려주시고 피자 크기가 커서 만족스럽습니다 맛있게 잘 먹었습니다,"'음식이 맛있어요', '재료가 신선해요', '친절해요', '매장이 청결해요'",1


### *3. DataLoader*

In [14]:
# 데이터로드를 위해 loader 객체 생성
loader = DataFrameLoader(sample_df, page_content_column="index")
docs = loader.load()
docs

[Document(page_content='37373', metadata={'user_name': '23499', 'place_review': '편하게 후다닥 \n유모차 가져가기도 좋구 ', 'place_tag': "'매장이 넓어요', '주차하기 편해요', '아이와 가기 좋아요', '음식이 빨리 나와요'", 'visit_count': 1}),
 Document(page_content='27382', metadata={'user_name': '4sik9DAD', 'place_review': '역시나 시장 \n값좋고 양도 많고 신선하고 좋네요\n마트보다 전통시장입니다\n\n 더 자세한 리뷰는 인스타그램 4sik9story 에서 확인하실 수 있어요\n인스타 오셔서 맞팔도 하시고 소통하며 함께해요', 'place_tag': "'신선해요', '양이 많아요', '친절해요', '가성비가 좋아요', '주차하기 편해요'", 'visit_count': 1}),
 Document(page_content='34283', metadata={'user_name': 'Acecream', 'place_review': '여름엔 동치미국수도 별미네요 \n비빔국수랑 고기만두까지 함께하니 배도 부르고\n간단하면서도 맛있게 한끼 했네요', 'place_tag': "'매장이 청결해요', '음식이 맛있어요', '특별한 메뉴가 있어요', '가성비가 좋아요'", 'visit_count': 8}),
 Document(page_content='14514', metadata={'user_name': 'Apricot92', 'place_review': '아아 맛도 가격도 굿굿', 'place_tag': "'음료가 맛있어요', '특별한 메뉴가 있어요', '친절해요'", 'visit_count': 4}),
 Document(page_content='26000', metadata={'user_name': 'AziA', 'place_review': '고기도 구워주구 맛있구 \n아쉽게도 양이 적으니 추가 추가요', 'p

### 4. TextSplitter

In [15]:
# 토큰제한으로 문자 split해야함.
character_text_spliter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits_docs = character_text_spliter.split_documents(docs)
splits_docs

[Document(page_content='37373', metadata={'user_name': '23499', 'place_review': '편하게 후다닥 \n유모차 가져가기도 좋구 ', 'place_tag': "'매장이 넓어요', '주차하기 편해요', '아이와 가기 좋아요', '음식이 빨리 나와요'", 'visit_count': 1}),
 Document(page_content='27382', metadata={'user_name': '4sik9DAD', 'place_review': '역시나 시장 \n값좋고 양도 많고 신선하고 좋네요\n마트보다 전통시장입니다\n\n 더 자세한 리뷰는 인스타그램 4sik9story 에서 확인하실 수 있어요\n인스타 오셔서 맞팔도 하시고 소통하며 함께해요', 'place_tag': "'신선해요', '양이 많아요', '친절해요', '가성비가 좋아요', '주차하기 편해요'", 'visit_count': 1}),
 Document(page_content='34283', metadata={'user_name': 'Acecream', 'place_review': '여름엔 동치미국수도 별미네요 \n비빔국수랑 고기만두까지 함께하니 배도 부르고\n간단하면서도 맛있게 한끼 했네요', 'place_tag': "'매장이 청결해요', '음식이 맛있어요', '특별한 메뉴가 있어요', '가성비가 좋아요'", 'visit_count': 8}),
 Document(page_content='14514', metadata={'user_name': 'Apricot92', 'place_review': '아아 맛도 가격도 굿굿', 'place_tag': "'음료가 맛있어요', '특별한 메뉴가 있어요', '친절해요'", 'visit_count': 4}),
 Document(page_content='26000', metadata={'user_name': 'AziA', 'place_review': '고기도 구워주구 맛있구 \n아쉽게도 양이 적으니 추가 추가요', 'p

### 5. Embedding, Vectorstore

In [16]:
# 임베딩 후 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=splits_docs, embedding=HuggingFaceBgeEmbeddings())

# 벡터스토어 내 벡터들 간 유사도 측정
retriever = vectorstore.as_retriever()

c:\Users\hare0\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\hare0\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hare0\.cache\huggingface\hub\models--BAAI--bge-large-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### 6. Prompt

In [17]:
prompt = ''' Classify the text into negative or positive.
review : {place_review}
classification : {classification}

place_review를 읽고 사용자의 리뷰가 긍정 혹은 부정인지 평가해줘.
만약 긍정이라면 1로 라벨링을 하고, 부정이라면 0으로 라벨링해줘.

- 긍정의 예시를 보여줄게.
'콩나물국밥 시원해요 김치도 맛있네요'는 긍정이야.
대체로 '맛있다', '또 가고 싶다','항상', '자주', '인생' 처럼 긍정적인 문맥이면 긍정으로 평가해줘.
또, visit_count(재방문 횟수)가 많을수록 긍정으로 평가해줘.

- 부정의 예시를 보여줄게.
'아이스 아메리카노 너무 묽어서 재구매는 안 할꺼에요'、‘다시는 안갈거에요’ 는 부정이야.
대체로 '실망이다', '맛없다', '다시는 안갈듯', '짜증','재방문은 글쎄','재수','불편','그닥' 처럼 부정적인 문맥이면 부정으로 평가해줘.'''

prompt_template = ChatPromptTemplate.from_template(prompt)

### 7. Model 생성

In [18]:
agent_gpt = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0.0, verbose=False, openai_api_key = os.environ['OPEN_API_KEY'])

lagchain = (
    {'classification':retriever, 'place_review':RunnablePassthrough()}
    | prompt_template #프롬프트
    | agent_gpt
    | StrOutputParser()
)

### 8. 긍정 부정 데이터 생성

In [19]:
classification_list = []

for document in docs:
  item_dict = {}
  item_dict['index'] = document.page_content
s
  for key, value in document.metadata.items():
    item_dict[key] = value

  # 긍정부정
  item_dict['classification'] = lagchain.invoke(item_dict['place_review'])
  classification_list.append(item_dict)


df_prompt_data = pd.DataFrame(classification_list)
df_prompt_data

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

> 프롬프트를 잘못 작성한 것 같다. <br>
> 프롬프트에 추가할 것 : 'classification 에는 1,0만 써줘', '이유는 쓰지마'

In [ ]:
for i in df_prompt_data['classification']:
  print(i)

[Document(page_content='1185', metadata={'user_name': 'OpotatoO', 'place_review': '요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고  같이간 많이가는 단골소녀가 애견동반 은 전화 문의후 방문주세요 반려동물은 전화로 문의후 소형견가방과함께있으면 가능하다고 하셔서  데리고갔습니다 요거트가 넘 맛있어서 딸기우유랑 무가당요거트 사왔어요  무가당은 강\n내용 더보기', 'place_tag': "'커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 있어요', '메뉴 구성이 알차요'", 'visit_count': 1})] -> Positive (1)

[Document(page_content='1233', metadata={'user_name': 'samestage', 'place_review': '용산 놀러왔다가 남편이 급으로 찾아낸 카페예요\n분위기도좋고 커피 찐맛입니다', 'place_tag': "'커피가 맛있어요', '인테리어가 멋져요', '사진이 잘 나와요', '아늑해요', '친절해요'", 'visit_count': 1})] -> Positive (1)

[Document(page_content='331', metadata={'user_name': '스트로베리홍시', 'place_review': '맛있어요옹 ㅜㅜㅜㅜ', 'place_tag': "'음식이 맛있어요', '양이 많아요'", 'visit_count': 1})] -> Positive (1)

[Document(page_content='809', metadata={'user_name': '울랄라언니', 'place_review': '계란장과 포케 조합 끝내주네요', 'place_tag': "'음식이 맛있어요', '재료가 신선해요'", 'visit_count': 1})] -> Positive (1)
1
1
Document(page_content='606', metadata={'user_name': '외계인의별', 

> 결과 요약 <br>
> Document(page_content='1185'): 1 <br>
> Document(page_content='1233') : 1 <br>
> Document(page_content='331') : 1 <br>
> Document(page_content='809') : 1 <br>
> Document(page_content='606') : 1 <br>
> Document(page_content='331) :1 <br>
> Document(page_content='452') : 1 <br>
> Document(page_content='37') : 1 <br>
> Document(page_content='606') : 1 <br>


In [ ]:
df_prompt_data['labeling'] = 1

In [ ]:
df_prompt_data[['place_review','classification', 'labeling']]

,place_review,classification,labeling
0,모임장소로 좋아요,"[Document(page_content='1185', metadata={'user...",1
1,방이먹자골목에서 디저트 찾으신다면 코지 카페에서 젤라또 드셔보시는거 추천드려요\n평...,1,1
2,요거트 꾸덕하니 너무너무 맛있었어요 젖소 키우신다고 같이간 많이가는 단골소녀가 애...,1,1
3,한남도 산책하다 우연히 발견한 카페 티라미슈로 알려진 곳인 것 같은데 아메리카노도 ...,"Document(page_content='606', metadata={'user_n...",1
4,용산 놀러왔다가 남편이 급으로 찾아낸 카페예요\n분위기도좋고 커피 찐맛입니다,"긍정적인 리뷰 예시:\n- ""분위기도 좋고 커피 찐맛입니다""\n- ""맛있어요옹""\n...",1
5,친절하고 맛있음\n주중에는 점심저녁 손님이 많아서 주말이 오히려 한가함,"리뷰 ""친절하고 맛있음 주중에는 점심저녁 손님이 많아서 주말이 오히려 한가함""은 긍...",1
6,맛있어요옹 ㅜㅜㅜㅜ,"리뷰 ""맛있어요옹 ㅜㅜㅜㅜ""는 긍정적인 문맥이므로 1로 라벨링합니다.",1
7,출장 중 라면 땡겨서 검색했다가\n1인 혼밥 가능한 항아리문어라면 있어서 먹으러왔어...,부정,1
8,계란장과 포케 조합 끝내주네요,"Document(page_content='331', metadata={'user_n...",1


> 문제 : 부정적인 리뷰 수가 적다.

****************